
<h2>Nom et Prenom : El ghazi Loubna</h2>


In [44]:
#Make sure to install :

#!pip3 install scrapy
#!pip3 install pymongo
#!pip3 install nltk
#!pip3 install requests
#!pip3 install spacy
#!python -m spacy download xx

In [1]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient #pour la connection a mongodb
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize


In [2]:
#1-Scraping from ALJAZERA :news WEBSITE

url = "https://www.aljazeera.net/news"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
# Find all article elements
articles = soup.find_all('article')
# Extract title, link, and summary for each article
arabic_articles = []
for article in articles:
    title = article.find('h3').text
    link = article.find('a')['href']
    summary = article.find('p').text
    arabic_articles.append({'title': title, 'link': link, 'summary': summary})

In [3]:
#2-Storing Data in MongoDB (nosql)

client = MongoClient('mongodb://localhost:27017/') # change yur port here
db = client['arabic_news']
articles_collection = db['articles']

articles_collection.insert_many(arabic_articles)

InsertManyResult([ObjectId('6612d000cd2a0cb6e46dffb7'), ObjectId('6612d000cd2a0cb6e46dffb8'), ObjectId('6612d000cd2a0cb6e46dffb9'), ObjectId('6612d000cd2a0cb6e46dffba'), ObjectId('6612d000cd2a0cb6e46dffbb'), ObjectId('6612d000cd2a0cb6e46dffbc'), ObjectId('6612d000cd2a0cb6e46dffbd'), ObjectId('6612d000cd2a0cb6e46dffbe'), ObjectId('6612d000cd2a0cb6e46dffbf'), ObjectId('6612d000cd2a0cb6e46dffc0'), ObjectId('6612d000cd2a0cb6e46dffc1'), ObjectId('6612d000cd2a0cb6e46dffc2'), ObjectId('6612d000cd2a0cb6e46dffc3'), ObjectId('6612d000cd2a0cb6e46dffc4')], acknowledged=True)

In [13]:
import spacy
import os
from nltk.stem import ISRIStemmer

####connectionn:
client = MongoClient('mongodb://localhost:27017/')
db = client['arabic_news']
articles_collection = db['articles']

###########################
nlp = spacy.blank('ar') #arbic using spacy

# Text cleaning
def clean_text(text):
    if os.path.exists(text):
        # Read the contents of the file
        with open(text, 'r', encoding='utf-8') as file:
            text = file.read()
    if '<' in text and '>' in text:
        # If it's HTML markup, u parse and extract text
        text = BeautifulSoup(text, 'html.parser').get_text()
    text = ''.join(c for c in text if c in 'اأإآبتثجحخدذرزسشصضطظعغفقكلمنهويةئ ')
    
    return text
    
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('arabic'))
    return [token for token in tokens if token not in stop_words]

def normalize(tokens):
    stemmer = ISRIStemmer()
    return [stemmer.stem(token) for token in tokens]

def nlp_pipeline(text):
    
    cleaned_text = clean_text(text)

    doc = nlp(cleaned_text)
    
    tokens = [token.text for token in doc]
    stop_words = set(stopwords.words('arabic'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    stemmer = ISRIStemmer()
    normalized_tokens = [stemmer.stem(token) for token in filtered_tokens]
    
    return normalized_tokens
    

In [14]:
#just a test 
example_text = "قامت المنتخبات الأربعة بالتسجيل لدى لجنة المباريات لبطولة كأس العالم 2022 في قطر."
normalized_tokens = nlp_pipeline(example_text)
print("Tokens normalisés:", normalized_tokens)

Tokens normalisés: ['قمت', 'نخب', 'ربع', 'سجل', 'لد', 'لجن', 'بري', 'بطل', 'كأس', 'علم', ' ', 'قطر']


In [42]:
arabic_articles = articles_collection.find()

for article in arabic_articles:
    normalized_tokens = nlp_pipeline(article['summary'])  # nb: I chosen summary
    
    articles_collection.update_one(
        {'_id': article['_id']},
        {'$set': {
            'normalized_tokens': normalized_tokens  
        }}
    )


<h3>Do the Stemming and Lemmatization, then compare the tow mechanisms. :Just the algorithm, we alreday done this approch on normalisation function</h3>

In [24]:
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
# Print the count of articles before entering the loop
#bc i have trouble to print them 
article_count = articles_collection.count_documents({})
print("Total number of articles in arabic_articles collection:", article_count)
#to compare the two mechanisms :
for article in arabic_articles:
    normalized_tokens = article.get('normalized_tokens', [])
    print("Normalized Tokens:", normalized_tokens)
    
    if not normalized_tokens:
        print("No normalized tokens found for this article.")
        continue
    
    try:
        # Stemming
        stemmed_tokens = stemming(normalized_tokens)
        
        # Lemmatization
        lemmatized_tokens = lemmatization(normalized_tokens)
        
        # Compare the two mechanisms
        print("Stemmed Tokens:", stemmed_tokens[:5])
        print("Lemmatized Tokens:", lemmatized_tokens[:5])
        print("---------------------------------------------------")
    except Exception as e:
        print("Error processing normalized tokens:", e)


Total number of articles in arabic_articles collection: 56


In [43]:
#we can see that lematized doosn t consider spaces as words !

<h3>Apply Parts of Speech technics based on both Rule based and Machine learning approaches.</h3>

In [22]:
#  Speech technics based on both Rule based and Machine learning approaches.
from nltk import pos_tag
# Example Arabic text :test
arabic_text = "قامت المنتخبات الأربعة بالتسجيل لدى لجنة المباريات لبطولة كأس العالم 2022 في قطر."
arabic_tokens = word_tokenize(arabic_text)

# Perform POS tagging
arabic_pos_tags = pos_tag(arabic_tokens)
print("Rule-Based - NLTK:", arabic_pos_tags)


Rule-Based - NLTK: [('قامت', 'JJ'), ('المنتخبات', 'NNP'), ('الأربعة', 'NNP'), ('بالتسجيل', 'NNP'), ('لدى', 'NNP'), ('لجنة', 'NNP'), ('المباريات', 'NNP'), ('لبطولة', 'NNP'), ('كأس', 'NNP'), ('العالم', 'NNP'), ('2022', 'CD'), ('في', 'NNP'), ('قطر', 'NNP'), ('.', '.')]


In [23]:
#On our data :

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['arabic_news']
articles_collection = db['articles']

# Retrieve articles from the collection
arabic_articles = articles_collection.find()
for article in arabic_articles:
    text = article.get('summary', '')
    
    # Tokenize the Arabic text into words
    arabic_tokens = word_tokenize(text)
    
    # Perform POS tagging
    arabic_pos_tags = pos_tag(arabic_tokens)
    
    # Print POS tags for the article
    print("Article Title:", article.get('title'))
    print("POS Tags (Rule-Based - NLTK):", arabic_pos_tags)
    print("---------------------------------------------------")

Article Title: الحركة الإسلامية بالقدس: الخطر القادم على الأقصى شديد
POS Tags (Rule-Based - NLTK): [('دعت', 'JJ'), ('الحركة', 'NNP'), ('في', 'NNP'), ('بيان', 'NNP'), ('تلقت', 'NNP'), ('“', 'NNP'), ('الجزيرة', 'NNP'), ('نت', 'NNP'), ('”', 'NNP'), ('نسخة', 'NNP'), ('منه', 'NNP'), ('إلى', 'NNP'), ('استمرار', 'NNP'), ('الرباط', 'NNP'), ('في', 'NNP'), ('المسجد', 'NNP'), ('الأقصى', 'NNP'), ('بالتزامن', 'NNP'), ('مع', 'NNP'), ('دعوات', 'NNP'), ('جماعات', 'NNP'), ('استيطانية', 'NNP'), ('لتكثيف', 'NNP'), ('اقتحامه', 'NNP'), ('.', '.')]
---------------------------------------------------
Article Title: نتنياهو: لا وقف إطلاق نار دون استعادة المحتجزين
POS Tags (Rule-Based - NLTK): [('قال', 'JJ'), ('رئيس', 'NNP'), ('الوزراء', 'NNP'), ('الإسرائيلي', 'NNP'), ('بنيامين', 'NNP'), ('نتنياهو', 'NNP'), ('-اليوم', 'NNP'), ('الأحد-', 'JJ'), ('إنه', 'NNP'), ('أوضح', 'NNP'), ('للمجتمع', 'NNP'), ('الدولي', 'NNP'), ('أنه', 'NNP'), ('لن', 'NNP'), ('يكون', 'NNP'), ('هناك', 'NNP'), ('وقف', 'NNP'), ('إطلاق', 'NNP')

In [30]:
#the machine learning-based POS tagging

#the pre-trained spaCy model for multilingual (so for arabic too)
nlp = spacy.load('xx_ent_wiki_sm')
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['arabic_news']
articles_collection = db['articles']

# Retrieve articles from the collection
arabic_articles = articles_collection.find()
# Iterate over each article and perform POS tagging
for article in arabic_articles:
    
    text = article.get('summary', '') 
    doc = nlp(text)
    arabic_pos_tags = [(token.text, token.pos_) for token in doc]

    print("Article Title:", article.get('title'))
    print("POS Tags :Machine Learning Approach :", arabic_pos_tags)
    print("---------------------------------------------------")

Article Title: الحركة الإسلامية بالقدس: الخطر القادم على الأقصى شديد
POS Tags :Machine Learning Approach : [('دعت', ''), ('الحركة', ''), ('في', ''), ('بيان', ''), ('تلقت', ''), ('“', ''), ('الجزيرة', ''), ('نت', ''), ('”', ''), ('نسخة', ''), ('منه', ''), ('إلى', ''), ('استمرار', ''), ('الرباط', ''), ('في', ''), ('المسجد', ''), ('الأقصى', ''), ('بالتزامن', ''), ('مع', ''), ('دعوات', ''), ('جماعات', ''), ('استيطانية', ''), ('لتكثيف', ''), ('اقتحامه', ''), ('.', '')]
---------------------------------------------------
Article Title: نتنياهو: لا وقف إطلاق نار دون استعادة المحتجزين
POS Tags :Machine Learning Approach : [('قال', ''), ('رئيس', ''), ('الوزراء', ''), ('الإسرائيلي', ''), ('بنيامين', ''), ('نتنياهو', ''), ('-اليوم', ''), ('الأحد-', ''), ('إنه', ''), ('أوضح', ''), ('للمجتمع', ''), ('الدولي', ''), ('أنه', ''), ('لن', ''), ('يكون', ''), ('هناك', ''), ('وقف', ''), ('إطلاق', ''), ('للنار', ''), ('في', ''), ('قطاع', ''), ('غزة', ''), ('،', ''), ('دون', ''), ('عودة', ''), ('المحتجزين', 

In [31]:
#Note : the POS tags obtained using the rule-based approach (NLTK) and the machine learning-based approach (spaCy) for the same text show differences in the tags assigned to certain words


<h3>6-NER Methods.</h3>

In [33]:
#test :
from nltk import ne_chunk


arabic_text = "قامت المنتخبات الأربعة بالتسجيل لدى لجنة المباريات لبطولة كأس العالم 2022 في قطر."

arabic_tokens = word_tokenize(arabic_text)

arabic_pos_tags = pos_tag(arabic_tokens)

# Perform NER using NLTK's ne_chunk
arabic_ner_tags = ne_chunk(arabic_pos_tags)

print("NER Tags (Rule-Based - NLTK):", arabic_ner_tags)

NER Tags (Rule-Based - NLTK): (S
  (ORGANIZATION قامت/JJ)
  (ORGANIZATION المنتخبات/NNP)
  الأربعة/NNP
  بالتسجيل/NNP
  لدى/NNP
  لجنة/NNP
  المباريات/NNP
  لبطولة/NNP
  كأس/NNP
  العالم/NNP
  2022/CD
  في/NNP
  قطر/NNP
  ./.)


In [38]:
#Now for our data :

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['arabic_news']
articles_collection = db['articles']
# Retrieve articles from the collection
arabic_articles = articles_collection.find()

for article in arabic_articles:
    article_text = article.get('summary', '') 
    arabic_tokens = word_tokenize(article_text)
    arabic_pos_tags = pos_tag(arabic_tokens)
    
    # NER
    arabic_ner_tags = ne_chunk(arabic_pos_tags)

    print("NER Tags for Article:", article['title'])
    print(arabic_ner_tags)
    print("---------------------------------------------------")

NER Tags for Article: الحركة الإسلامية بالقدس: الخطر القادم على الأقصى شديد
(S
  دعت/JJ
  (ORGANIZATION الحركة/NNP)
  في/NNP
  بيان/NNP
  تلقت/NNP
  “/NNP
  الجزيرة/NNP
  نت/NNP
  ”/NNP
  نسخة/NNP
  منه/NNP
  إلى/NNP
  استمرار/NNP
  الرباط/NNP
  في/NNP
  المسجد/NNP
  الأقصى/NNP
  بالتزامن/NNP
  مع/NNP
  دعوات/NNP
  جماعات/NNP
  استيطانية/NNP
  لتكثيف/NNP
  اقتحامه/NNP
  ./.)
---------------------------------------------------
NER Tags for Article: نتنياهو: لا وقف إطلاق نار دون استعادة المحتجزين
(S
  قال/JJ
  (ORGANIZATION رئيس/NNP)
  الوزراء/NNP
  الإسرائيلي/NNP
  بنيامين/NNP
  نتنياهو/NNP
  -اليوم/NNP
  الأحد-/JJ
  (ORGANIZATION إنه/NNP)
  أوضح/NNP
  للمجتمع/NNP
  الدولي/NNP
  أنه/NNP
  لن/NNP
  يكون/NNP
  هناك/NNP
  وقف/NNP
  إطلاق/NNP
  للنار/NNP
  في/NNP
  قطاع/NNP
  غزة،/NNP
  دون/NNP
  عودة/NNP
  المحتجزين،/NNP
  مضيفا/NNP
  أنه/NNP
  لن/NNP
  يرضخ/NNP
  لمطالب/NNP
  حركة/NNP
  حماس،/NNP
  وفق/NNP
  كلامه/NNP
  ./.)
---------------------------------------------------
NER Tags for

In [39]:
#spaCy (Machine Learning-Based):
#test
# Load the Arabic spaCy model
nlp = spacy.blank('ar')

# Example Arabic text
arabic_text = "قامت المنتخبات الأربعة بالتسجيل لدى لجنة المباريات لبطولة كأس العالم 2022 في قطر."

# Process the text using spaCy
doc = nlp(arabic_text)

# Extract named entities using spaCy
arabic_ner_tags = [(ent.text, ent.label_) for ent in doc.ents]

print("NER Tags (Machine Learning-Based - spaCy):", arabic_ner_tags)

NER Tags (Machine Learning-Based - spaCy): []


In [41]:
nlp = spacy.load('xx_ent_wiki_sm')  
client = MongoClient('mongodb://localhost:27017/')
db = client['arabic_news']
articles_collection = db['articles']
# Retrieve articles from the collection
arabic_articles = articles_collection.find()
for article in arabic_articles:
   
    article_text = article.get('summary', '')  
    doc = nlp(article_text)
    

    print("NER Tags :Machine Learning Approach for Article:", article['title'])
    for ent in doc.ents:
        print(ent.text, ent.label_)
    print("---------------------------------------------------")

NER Tags :Machine Learning Approach for Article: الحركة الإسلامية بالقدس: الخطر القادم على الأقصى شديد
دعت ORG
الجزيرة نت MISC
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: نتنياهو: لا وقف إطلاق نار دون استعادة المحتجزين
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: مستشار المرشد الإيراني يهدد السفارات الإسرائيلية وتل أبيب تعلن استعدادها
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: "عملية نوعية من الطراز الفاخر".. كمين القسام في الزنة يتصدر منصات التواصل
تفاعل MISC
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: كاتب إسرائيلي: حماس غيرت مسار التاريخ وقدمت إنجازا عبقريا في 6 أشهر
آلون مزراحي PER
---------------------------------------------------
NER Tags :Machine Learning Approach for Article: الأمراض المعدية تطارد الهاربين من الموت في غزة
قالت PER
----------------------